In [12]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

import scipy.io as sio
import pandas as pd
import numpy as np
from torch_geometric.data import Data

2.0.1+cpu


Das Proprocessing beruht auf: https://colab.research.google.com/drive/1EBV4eUqUoGpTYqBK2YgHvNtwLjxu1nnc?usp=sharing#scrollTo=Iv1GAxuPtkqI
Dataset Download: https://github.com/YingtongDou/CARE-GNN

In [13]:
# Laden des Datensatzes
data = sio.loadmat('Data/Amazon.mat', verify_compressed_data_integrity=False)
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Aug 19 20:20:49 2020',
 '__version__': '1.0',
 '__globals__': [],
 'homo': <11944x11944 sparse matrix of type '<class 'numpy.float64'>'
 	with 8796784 stored elements in Compressed Sparse Column format>,
 'net_upu': <11944x11944 sparse matrix of type '<class 'numpy.float64'>'
 	with 351216 stored elements in Compressed Sparse Column format>,
 'net_usu': <11944x11944 sparse matrix of type '<class 'numpy.float64'>'
 	with 7132958 stored elements in Compressed Sparse Column format>,
 'net_uvu': <11944x11944 sparse matrix of type '<class 'numpy.float64'>'
 	with 2073474 stored elements in Compressed Sparse Column format>,
 'features': <11944x25 sparse matrix of type '<class 'numpy.float64'>'
 	with 174488 stored elements in Compressed Sparse Column format>,
 'label': array([[0., 0., 0., ..., 0., 0., 0.]])}

In [14]:
# Label überprüfen
label = data['label']
print(label.shape)
print(label)
print(label.sum())

(1, 11944)
[[0. 0. 0. ... 0. 0. 0.]]
821.0


In [15]:
# Features überprüfen
features = data['features']
feature_data = pd.DataFrame.sparse.from_spmatrix(features)
print(feature_data.shape)
print(feature_data.head(5))

(11944, 25)
    0     1    2    3    4    5    6    7     8     9   ...   15   16   17  \
0  1.0  26.0  0.0  0.0  0.0  0.0  1.0  0.0  0.00  0.00  ...  5.0  5.0  5.0   
1  4.0  17.0  0.0  1.0  1.0  0.0  2.0  0.0  0.25  0.25  ...  4.0  5.0  2.0   
2  2.0  15.0  0.0  0.0  0.0  2.0  0.0  0.0  0.00  0.00  ...  4.0  4.0  4.0   
3  1.0  21.0  0.0  0.0  0.0  0.0  1.0  0.0  0.00  0.00  ...  5.0  5.0  5.0   
4  2.0  18.0  0.0  0.0  0.0  1.0  1.0  0.0  0.00  0.00  ...  4.5  5.0  4.0   

     18   19      20        21   22    23   24  
0  5.00  0.0     0.0  0.000000  1.0  13.0  1.0  
1  3.75  0.0  3382.0  1.386294  0.0  45.0  1.0  
2  4.00  0.0     0.0  0.000000  1.0  24.5  1.0  
3  5.00  0.0     0.0  0.000000  1.0  14.0  1.0  
4  4.50  0.0     0.0  0.000000  1.0  18.5  1.0  

[5 rows x 25 columns]


In [16]:
# Ermittlung des Kantenindex für drei Netze: upu, usu und uvu
net_upu_edges = data['net_upu'].nonzero()
net_usu_edges = data['net_usu'].nonzero()
net_uvu_edges = data['net_uvu'].nonzero()

In [17]:
## Verarbeitung des usu-Netzwerks
#  Filtert den usu-Graphen auf Knoten mit einem Grad ungleich Null.

# In dieser Liste ist der Index der neue Knotenindex und der Wert bei index ist der ursprüngliche Knotenindex.
non_zero_degree_nodes_idex = sorted(list(set(net_usu_edges[0].tolist())))

# Aufbau des Lookup-Diktats
old_index_to_new_index = dict()
for idx in range(len(non_zero_degree_nodes_idex)):
  old_index_to_new_index[non_zero_degree_nodes_idex[idx]] = idx

# Abrufen des neuen x-dataframes
usu_x = feature_data.iloc[non_zero_degree_nodes_idex]

# Neue Labels
usu_label = data['label'][0][non_zero_degree_nodes_idex]


# Kanten updaten
for i in range(2):
  for idx in range(net_usu_edges[0].shape[0]):
    net_usu_edges[i][idx] = old_index_to_new_index[net_usu_edges[i][idx]]

In [18]:
# Erzeugen des Train, Test und Validation Split
data_pop = np.arange(usu_label.size)
indices = np.random.permutation(data_pop.size)

train = int(round(0.6*usu_label.size,0))
test = int(round(0.2*usu_label.size,0))

train_idx = indices[:train]
val_idx = indices[train:train + test]
test_idx = indices[train + test:]

print(f"Train, validation, test split is {train_idx.size, val_idx.size, test_idx.size}")

train_array = np.empty([1, usu_label.size])
val_array = np.empty([1, usu_label.size])
test_array = np.empty([1, usu_label.size])


for value in data_pop:
  if value in train_idx:
    train_array[0][value] = True
  else:
    train_array[0][value] = False

for value in data_pop:
  if value in val_idx:
    val_array[0][value] = True
  else:
    val_array[0][value] = False

for value in data_pop:
  if value in test_idx:
    test_array[0][value] = True
  else:
    test_array[0][value] = False

train_final = np.array(train_array[0], dtype=bool)
val_final = np.array(val_array[0], dtype=bool)
test_final = np.array(test_array[0], dtype=bool)


train_mask = torch.Tensor(train_final).bool()
val_mask = torch.Tensor(val_final).bool()
test_mask = torch.Tensor(test_final).bool()

Train, validation, test split is (7112, 2371, 2371)


In [19]:
# Graphen aufsetzen
x = torch.tensor(usu_x.values, dtype = torch.float)
y = torch.tensor(usu_label, dtype = torch.long)

# Kantenindizes aufsetzen
edge_index = torch.tensor([net_usu_edges[0], net_usu_edges[1]], dtype=torch.long)

usu_graph = Data(x=x, y=y, edge_index=edge_index, train_mask = train_mask, val_mask = val_mask, test_mask = test_mask)
torch.save(usu_graph, 'Graph/usu_graph.pth')
usu_graph

Data(x=[11854, 25], edge_index=[2, 7132958], y=[11854], train_mask=[11854], val_mask=[11854], test_mask=[11854])

In [20]:
data = torch.load('Graph/usu_graph.pth')
data

Data(x=[11854, 25], edge_index=[2, 7132958], y=[11854], train_mask=[11854], val_mask=[11854], test_mask=[11854])